In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

MAIN_URL = "https://www.dhlottery.co.kr/gameResult.do?method=byWin"  # 메인 페이지 URL
BASIC_URL = "https://www.dhlottery.co.kr/gameResult.do?method=byWin&drwNo="  # 회차 정보 페이지 URL

def get_last_round():
    try:
        # 메인 페이지에서 마지막 회차 정보를 가져오는 함수 
        resp = requests.get(MAIN_URL)
        resp.raise_for_status()  # HTTP 오류가 발생하면 예외 발생
        soup = BeautifulSoup(resp.text, "html.parser")
        # 메타 태그에서 회차 정보 추출
        description = soup.find("meta", {"id": "desc", "name": "description"})['content']
        last_round = int(description.split()[1][:-1])
        return last_round
    except Exception as e:
        print("마지막 회차 정보를 가져오는 중 오류 발생:", e)
        return None

def get_lottery_info(round_number):
    try:
        # 특정 회차의 당첨번호와 당첨금 정보를 가져오는 함수
        url = BASIC_URL + str(round_number)
        resp = requests.get(url)
        resp.raise_for_status()  # HTTP 오류가 발생하면 예외 발생
        soup = BeautifulSoup(resp.text, "html.parser")

        text = soup.text

        s_idx = text.find(" 당첨결과")
        s_idx = text.find("당첨번호", s_idx) + 4
        e_idx = text.find("보너스", s_idx)
        numbers = text[s_idx:e_idx].strip().split()

        s_idx = e_idx + 3
        e_idx = s_idx + 3
        bonus = text[s_idx:e_idx].strip()

        prize_info = {}
        for i in range(1, 6):
            s_idx = text.find(f"{i}등", e_idx) + 2
            e_idx = text.find("원", s_idx) + 1
            e_idx = text.find("원", e_idx)
            prize_info[f"money{i}"] = text[s_idx:e_idx].strip().replace(',', '').split()[2]

        return numbers, bonus, prize_info
    except Exception as e:
        print("회차", round_number, "의 당첨정보를 가져오는 중 오류 발생:", e)
        return None, None, None

def write_to_csv(file_path, data):
    try:
        # CSV 파일에 데이터를 저장하는 함수
        with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(["회차", "1등번호", "2등번호", "3등번호", "4등번호", "5등번호", "6등번호", "보너스", "1등당첨금액", "2등당첨금액", "3등당첨금액", "4등당첨금액", "5등당첨금액"])
            writer.writerows(data)
    except Exception as e:
        print("CSV 파일에 데이터를 저장하는 중 오류 발생:", e)

def main():
    # 메인 함수
    last_round = get_last_round()  # 마지막 회차 정보 가져오기
    if last_round is not None:
        try:
            data = []
            for i in range(last_round -1  , last_round + 1):
                numbers, bonus, prize_info = get_lottery_info(i)  # 각 회차별 당첨정보 가져오기

                print(" 각 회차별 당첨정보 가져오기 ", i)
                    
                if numbers is not None and bonus is not None and prize_info is not None:
                    row = [i] + numbers + [bonus] + list(prize_info.values())
                    data.append(row)
            write_to_csv('2020-1-25-keras_lstm_lotto_v895_data.csv', data)  # CSV 파일에 저장
        except Exception as e:
            print("주요 작업을 실행하는 중 오류 발생:", e)

if __name__ == "__main__":
    main()


In [ ]:
#sudo apt install python3-ipykernel
#sudo apt install python3-matplotlib
#sudo apt install python3.11-venv

#python3 -m venv .venv

#source ~/.venv/bin/activate
#deactivate
